In [1]:
import numpy as np
import struct

In [2]:
sbox = lambda a: np.array((
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,            
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
))[a.reshape(-1)].reshape(4, 4)

In [3]:
inv_sbox = lambda b: np.array((
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
))[b.reshape(-1)].reshape(4, 4)

In [4]:
shift_rows = lambda b: (
    (b[0][0], b[0][1], b[0][2], b[0][3]),
    (b[1][1], b[1][2], b[1][3], b[1][0]),
    (b[2][2], b[2][3], b[2][0], b[2][1]),
    (b[3][3], b[3][0], b[3][1], b[3][2]),
)

In [5]:
inv_shift_rows = lambda b: np.array((
    (b[0][0], b[0][1], b[0][2], b[0][3]),
    (b[1][3], b[1][0], b[1][1], b[1][2]),
    (b[2][2], b[2][3], b[2][0], b[2][1]),
    (b[3][1], b[3][2], b[3][3], b[3][0]),
))

In [6]:
xtime = lambda b: np.where(
    np.bitwise_and(b, 0x80), 
    np.bitwise_and(np.bitwise_xor(np.left_shift(b, 1), 0x1B), 0xFF),
    np.left_shift(b, 1)
)

In [7]:
def mix_columns(b):
    t = np.bitwise_xor(np.bitwise_xor(b[0], b[1]), np.bitwise_xor(b[2], b[3]))
    c = np.zeros((4, 4))
    c[0] = np.bitwise_xor(b[0], np.bitwise_xor(t, xtime(np.bitwise_xor(b[0], b[1]))))
    c[1] = np.bitwise_xor(b[1], np.bitwise_xor(t, xtime(np.bitwise_xor(b[1], b[2]))))
    c[2] = np.bitwise_xor(b[2], np.bitwise_xor(t, xtime(np.bitwise_xor(b[2], b[3]))))
    c[3] = np.bitwise_xor(b[3], np.bitwise_xor(t, xtime(np.bitwise_xor(b[3], b[0]))))
    return c.astype(np.int16)

In [8]:
def inv_mix_columns(c):
    u = xtime(xtime(np.bitwise_xor(c[0], c[2])))
    v = xtime(xtime(np.bitwise_xor(c[1], c[3])))
    b = np.zeros((4, 4))
    b[0] = np.bitwise_xor(c[0], u)
    b[1] = np.bitwise_xor(c[1], v)
    b[2] = np.bitwise_xor(c[2], u)
    b[3] = np.bitwise_xor(c[3], v)
    return mix_columns(b.astype(np.int16)).astype(np.int16)

In [9]:
diffusion = lambda b: mix_columns(shift_rows(b))

In [10]:
inv_diffusion = lambda c: inv_shift_rows(inv_mix_columns(c))

In [11]:
key_addition = lambda c, k: np.bitwise_xor(k, c)

In [12]:
inv_key_addition = lambda a, k: np.bitwise_xor(k, a)

In [13]:
g = lambda k:(
    (
        0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
        0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
        0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
        0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
        0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
        0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
        0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
        0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
        0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
        0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
        0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
        0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
        0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
        0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
        0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
        0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
    )[k[1]], 
    k[2], 
    k[3], 
    k[0],
)

In [14]:
def compute_keys(k):
    ks = []
    ks.append(k)
    for i in range(1, 11):
        gk3 = g(ks[i-1][3])
        k0 = (
            np.bitwise_xor(gk3[0], ks[i-1][0][0]),
            np.bitwise_xor(gk3[1], ks[i-1][0][1]),
            np.bitwise_xor(gk3[2], ks[i-1][0][2]),
            np.bitwise_xor(gk3[3], ks[i-1][0][3]),
        )
        k1 = (
            np.bitwise_xor(k0[0], ks[i-1][1][0]),
            np.bitwise_xor(k0[1], ks[i-1][1][1]),
            np.bitwise_xor(k0[2], ks[i-1][1][2]),
            np.bitwise_xor(k0[3], ks[i-1][1][3]),
        )
        k2 = (
            np.bitwise_xor(k1[0], ks[i-1][2][0]),
            np.bitwise_xor(k1[1], ks[i-1][2][1]),
            np.bitwise_xor(k1[2], ks[i-1][2][2]),
            np.bitwise_xor(k1[3], ks[i-1][2][3]),
        )
        k3 = (
            np.bitwise_xor(k2[0], ks[i-1][3][0]),
            np.bitwise_xor(k2[1], ks[i-1][3][1]),
            np.bitwise_xor(k2[2], ks[i-1][3][2]),
            np.bitwise_xor(k2[3], ks[i-1][3][3]),
        )
        k = np.array((k0, k1, k2, k3)).T
        ks.append(k)
    return tuple(ks)

In [15]:
def encrypt(x, k):
    x = np.array(list(x.to_bytes(16, 'big'))).reshape(4, 4)
    k = np.array(list(k.to_bytes(16, 'big'))).reshape(4, 4)
    ks = compute_keys(k)
    y = key_addition(x, ks[0])
    for i in range(1, 10):
        y = sbox(y)
        y = diffusion(y)
        y = key_addition(y, ks[i])
    y = sbox(y)
    y = shift_rows(y)
    a = key_addition(y, ks[10]).reshape(-1)
    y = 0
    for i in range(16):
        y += int(a[i]) * 256 ** (15 - i)
    return y

In [16]:
def decrypt(y, k):
    y = np.array(list(y.to_bytes(16, 'big'))).reshape(4, 4)
    k = np.array(list(k.to_bytes(16, 'big'))).reshape(4, 4)
    ks = compute_keys(k)
    x = inv_key_addition(ks[10], y)
    x = inv_shift_rows(x)
    x = inv_sbox(x)
    for i in range(9, 0, -1):
        x = inv_key_addition(ks[i], x)
        x = inv_diffusion(x)
        x = inv_sbox(x)
    a = inv_key_addition(x, ks[0]).reshape(-1)
    x = 0
    for i in range(16):
        x += int(a[i]) * 256 ** (15 - i)
    return x